In [234]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import fastText
ft_model = fastText.load_model('../eml2vec/wiki.ru/wiki.ru.bin')

In [270]:
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
bad_pos = {'CONJ', 'PREP', 'NPRO', 'PRCL'}

def clean_text(text):
    text = re.sub(r'[^\w\s]',' ', text)
    text = text.lower()
    clean_text = []
    for word in text.split():
        if not word:
            continue
#         if morph.parse(word)[0].tag.POS in bad_pos:
#             continue
        clean_text.append(word)
    return ' '.join(clean_text)

In [271]:
q2a = []
with open('faq.txt') as faq_input:
    for i, line in enumerate(faq_input):
        if i % 2 == 0:
            q2a.append([line.rstrip()])
        else:
            q2a[-1].append(line.rstrip())

In [285]:
df = pd.DataFrame(data=q2a, columns=['q_raw', 'a'])
df['q_a'] = df.apply(lambda row: row['q_raw'] + ' ' + row['a'], axis=1)
df['q'] = df['q_raw'].map(clean_text)
df['a'] = df['a'].map(clean_text)
df['q_a'] = df['q_a'].map(clean_text)

In [286]:
def get_sentence_vector(sentence):
    vec = np.zeros_like(ft_model.get_word_vector('привет'))
    sentence_split = sentence.split() 
    for word in sentence.split():
        vec += ft_model.get_word_vector(word) / len(sentence_split)
    return vec

In [353]:
df['q_vec'] = df['q'].map(ft_model.get_sentence_vector)
df['q_a_vec'] = df['q_a'].map(ft_model.get_sentence_vector)
df['q_vec'] = df['q'].map(ft_model.get_sentence_vector)
df['q_a_vec'] = df['q_a'].map(ft_model.get_sentence_vector)
df['q_a_w_vec'] = df.apply(lambda row: list(ft_model.get_sentence_vector(row['q']) * 0.2 + 
                                              ft_model.get_sentence_vector(row['a']) * 0.8), axis=1)
df['q_vec_unnormed'] = df['q'].map(get_sentence_vector)
df['q_a_vec_unnormed'] = df['q_a'].map(get_sentence_vector)
df['q_a_w_vec_unnormed'] = df.apply(lambda row: list(get_sentence_vector(row['q']) * 0.2 + 
                                                      get_sentence_vector(row['a']) * 0.8), axis=1)

df['q_a_concat_vec_unnormed'] = df.apply(lambda row: 
                                               list(np.hstack((get_sentence_vector(row['q']),
                                                               get_sentence_vector(row['a']) / 4))), axis=1)

In [354]:
df_test = pd.DataFrame(
    data=[['как уйти в отпуск', 'Как запланировать отпуск?', 'Как запланировать отпуск?'],
          ['компенсируйте мне отпуск пожалуйста', 'сделай мне одолжение', 'Возможна ли денежная компенсация за неиспользованный отпуск?'],
          ['можно ли оформлять отпуск на выходные', 'Можно ли планировать отпуск на выходные дни?', 'Можно ли планировать отпуск на выходные дни?'],
          ['как сделать пропуск', 'Когда требуется оформление пропуска и как это сделать?', 'Когда требуется оформление пропуска и как это сделать?'],
          ['можно ли не ходить в отпуск', 'Обязательно ли сотруднику идти в отпуск?', 'Обязательно ли сотруднику идти в отпуск?'],
          ['здравствуйте', 'привет привет', 'привет привет'],
          ['как получить зарплатную карту', 'В каком порядке выдаются зарплатные карты?', 'В каком порядке выдаются зарплатные карты?'],
          ['можно ли сделать больничный в отпуске', 'Обязательно ли сотруднику идти в отпуск?', 'Если во время отпуска работник заболел, получил листок нетрудоспособности, можно ли перенести, то есть «догулять» положенные дни. Если да — как это сделать и переносятся ли эти дни на следующий год?'],
          ['как получить копию трудового договора', 'Когда и как можно забрать свой экземпляр трудового договора?', 'Когда и как можно забрать свой экземпляр трудового договора?'],
          ['за сколько дней нужно подавать заявку на отпуск?', 'Как запланировать отпуск?', 'За сколько дней можно оформить отпуск?'],
          ['сколько дней отпуска можно взять единовременно', 'Какое количество дней отпуска можно взять (мин/макс)?', 'Какое количество дней отпуска можно взять (мин/макс)?'],
          ['когда выдаются премии?', 'Где можно подробнее ознакомиться с правилами годового премирования?', 'Где можно подробнее ознакомиться с правилами годового премирования?'],
          ['что делать если я заболел', 'Какой порядок вынесения работнику дисциплинарного взыскания?', 'Как оформляется больничный лист'],
          ['отменяются ли дни отпуска, которые пересекаются с больничным?', 'Если во время отпуска работник заболел, получил листок нетрудоспособности, можно ли перенести','Если во время отпуска работник заболел'],
          ['можно ли оформить отпуск на выходные?', 'Можно ли планировать отпуск на выходные дни?', 'Можно ли планировать отпуск на выходные дни?'],
          ['даются ли отгулы за сдачу крови?', 'Возможна ли денежная компенсация за неиспользованный отпуск?', 'Отпуск для доноров: сколько дней полагается и как они оплачиваются?'],
          ['есть ли дополнительный отпуск тем, кто работал в Чернобыле?', 'ты кто', 'Как оформяется отпуск чернобыльцам?']
         ],
        columns=['user_q', 'q_autof', 'q_best'])

df_test['user_q_vec'] = df_test['user_q'].map(clean_text).map(ft_model.get_sentence_vector)
df_test['user_q_vec_unnormed'] = df_test['user_q'].map(clean_text).map(get_sentence_vector)
df_test['user_q_concat_vec_unnormed'] = df_test['user_q'].map(clean_text)\
    .map(get_sentence_vector).map(lambda x: np.hstack((x,x)))

In [355]:
len(df_test['user_q_concat_vec_unnormed'].iloc[0])

600

In [356]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1, weights='distance')

def get_nearest(neighborhood, user_queries):
    knn.fit(np.vstack(neighborhood.values), np.zeros(len(neighborhood)))
    knn_result = knn.kneighbors(np.vstack(user_queries.values), n_neighbors=1)
    return df['q_raw'][knn_result[1][:,0]], knn_result[0][:,0]

experiments = {}

experiments['q'] = get_nearest(df['q_vec'], df_test['user_q_vec'])
experiments['q_a'] = get_nearest(df['q_a_vec'], df_test['user_q_vec'])
experiments['q_a_w'] = get_nearest(df['q_a_w_vec'], df_test['user_q_vec'])
experiments['q_unnormed'] = get_nearest(df['q_vec_unnormed'], df_test['user_q_vec_unnormed'])
experiments['q_a_unnormed'] = get_nearest(df['q_a_vec_unnormed'], df_test['user_q_vec_unnormed'])
experiments['q_a_w_unnormed'] = \
    get_nearest(df['q_a_w_vec_unnormed'], df_test['user_q_vec_unnormed'])
experiments['q_a_concat_unnormed'] = \
    get_nearest(df['q_a_concat_vec_unnormed'], df_test['user_q_concat_vec_unnormed'])

In [357]:
from collections import defaultdict
experiment2score = defaultdict(int)

def is_match_correct(match, best_match):
    if match.startswith(best_match):
        return '+'
    return '-'


for i, (user_q, q_best, q_autof) in enumerate(zip(df_test['user_q'], df_test['q_best'], df_test['q_autof'])):
    print('User:'.ljust(22), user_q)
    print('Best Match:'.ljust(22), q_best)
    print('AutoF:'.ljust(20), is_match_correct(q_autof, q_best), q_autof)
    if is_match_correct(q_autof, q_best) == '+':
        experiment2score['AutoF'] += 1
    
    for exper_name, (q_matches, distances) in experiments.items():
        q_match, distance = q_matches.iloc[i], distances[i]
        print(exper_name.ljust(20), is_match_correct(q_match, q_best), q_match, round(distance, 4))
        if is_match_correct(q_match, q_best) == '+':
            experiment2score[exper_name] += 1
    print()
    
    
print('\nScores ({} questions):'.format(len(df_test)))
for exper_name, score in experiment2score.items():
    print(score, exper_name)

User:                  как уйти в отпуск
Best Match:            Как запланировать отпуск?
AutoF:               + Как запланировать отпуск?
q                    + Как запланировать отпуск? 0.3848
q_a                  - Как оформяется отпуск чернобыльцам? 0.4067
q_a_w                - Как оформяется отпуск чернобыльцам? 0.3916
q_unnormed           + Как запланировать отпуск? 1.5488
q_a_unnormed         - Как оформяется отпуск чернобыльцам? 1.5606
q_a_w_unnormed       - Как оформяется отпуск чернобыльцам? 1.5179
q_a_concat_unnormed  + Как запланировать отпуск? 2.6557

User:                  компенсируйте мне отпуск пожалуйста
Best Match:            Возможна ли денежная компенсация за неиспользованный отпуск?
AutoF:               - сделай мне одолжение
q                    - Могу ли я добавить дополнительного согласователя при планировании отпуска? 0.5204
q_a                  - уточни о чём ты 0.4921
q_a_w                - уточни о чём ты 0.4882
q_unnormed           - За сколько дней можно